TODO: 
Clean Strings
...


In [1]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize
import time, random, re, pprint
pd.set_option('display.max_columns', None)
print(sys.version_info)

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


## Reading Data and Cleaning

In [119]:
%%time
start = time.time()
train_input = pd.read_csv('train_input.csv')
train_output = pd.read_csv('train_output.csv')

CPU times: user 1 s, sys: 198 ms, total: 1.2 s
Wall time: 1.34 s


In [120]:
print(train_input.shape)
train_input.head()

(165000, 2)


,id,conversation
0,0,<speaker_1> seaworld ceo steps down amid tanki...
1,1,<speaker_1> strickland chargers owner dean spa...
2,2,<speaker_1> iniesta plays keepy uppy with one ...
3,3,<speaker_1> chappie trailer #1 <number> hugh j...
4,4,<speaker_1> why the church of satan may get to...


In [126]:
delchars = ''.join(c for c in map(chr, range(256)) if not c.isalpha())
delchars = ''.join(ch for ch in delchars if ch !=' ')

def clean_text(paragraph):
    output = re.sub('\<.*?\>','', paragraph) #delete tags (between <..>)
    output = re.sub('\@.*?\s','', output) #delete usernames (words following @)
    output = re.sub('\n','', output) #delete anything following a slash
    output = output.translate(str.maketrans('','',delchars))
    return output

def rem_stopwords(word_list): 
    return list(set(word_list) - set(stopwords.words('English')))

count_stops = Counter(stopwords.words("English")*100)

def rem_stopwords_count(words_list):
    return Counter(words_list) - count_stops

def generate_features(dataframe):
    dataframe["text"] = dataframe["conversation"].apply(lambda x: clean_text(x))
    dataframe["words"] = dataframe["text"].apply(lambda x: x.split()) 
    dataframe["words_count"] = dataframe["words"].apply(lambda x: rem_stopwords_count(x))
    return dataframe

In [127]:
%%time
train_input = generate_features(train_input)
train_input["output"] = train_output["category"]
#strip out all common words (stopwords, which we import from the nltk package)

CPU times: user 21.7 s, sys: 2.25 s, total: 24 s
Wall time: 24.3 s


In [128]:
train_input.head()

,id,conversation,text,words,words_count,output
0,0,<speaker_1> seaworld ceo steps down amid tanki...,seaworld ceo steps down amid tanking revenues...,"[seaworld, ceo, steps, down, amid, tanking, re...","{'com': 1, 'good': 1, 'regarding': 1, 'propaga...",news
1,1,<speaker_1> strickland chargers owner dean spa...,strickland chargers owner dean spanos and gol...,"[strickland, chargers, owner, dean, spanos, an...","{'twitter': 1, 'sachs': 2, 'strickland': 1, 't...",nfl
2,2,<speaker_1> iniesta plays keepy uppy with one ...,iniesta plays keepy uppy with one leg man yo...,"[iniesta, plays, keepy, uppy, with, one, leg, ...","{'com': 1, 'man': 1, 'almost': 1, 'walking': 1...",soccer
3,3,<speaker_1> chappie trailer #1 <number> hugh j...,chappie trailer hugh jackman sci fi comedy...,"[chappie, trailer, hugh, jackman, sci, fi, com...","{'horrifically': 1, 'intentionally': 1, 'sad':...",movies
4,4,<speaker_1> why the church of satan may get to...,why the church of satan may get to open your ...,"[why, the, church, of, satan, may, get, to, op...","{'deities': 1, 'america': 1, 'next': 1, 'praye...",politics


In [7]:
train_input["words_count"][0]

Counter({'amid': 1,
         'animals': 1,
         'announces': 1,
         'blackfish': 1,
         'campaign': 1,
         'ceo': 1,
         'changes': 1,
         'com': 1,
         'company': 1,
         'couple': 1,
         'coupled': 1,
         'criticism': 1,
         'documentary': 1,
         'good': 1,
         'killer': 1,
         'layoffs': 1,
         'least': 1,
         'like': 1,
         'little': 1,
         'make': 1,
         'maybe': 2,
         'media': 1,
         'nothing': 1,
         'number': 1,
         'outsideonline': 1,
         'pathetic': 1,
         'perception': 1,
         'propaganda': 1,
         'public': 2,
         'quo': 1,
         'regarding': 1,
         'revenues': 1,
         'safety': 1,
         'saying': 1,
         'seaworld': 2,
         'status': 1,
         'steps': 1,
         'sway': 1,
         'tanking': 1,
         'trainers': 1,
         'treatment': 1,
         'underscored': 1,
         'undisclosed': 1,
         'uphel

# Now, Learn the Naive Bayes Model

In [171]:
train_input.groupby("output").size()
#Size of each of the groups

output
hockey       20861
movies       22409
nba          18422
news         21057
nfl          20106
politics     19694
soccer       21363
worldnews    21088
dtype: int64

In [9]:
##Recursive algorithm to efficiently sum a list of counters
def mergeSum(Counter_list):
    if len(Counter_list) < 2:
        return Counter_list.iloc[0]
    
    else: 
        mid = len(Counter_list)//2

        lefthalf = mergeSum(Counter_list[:mid])
        righthalf = mergeSum(Counter_list[mid:])

        my_sum = lefthalf + righthalf
        return my_sum

In [10]:
%%time
#Total word Counter
total_counter = mergeSum(train_input["words_count"])

CPU times: user 1min 11s, sys: 2.01 s, total: 1min 13s
Wall time: 1min 14s


In [11]:
%%time 
#Word Counter for each group
groups_counter = train_input.groupby("output")["words_count"].apply(lambda x: mergeSum(x))

CPU times: user 1min 11s, sys: 641 ms, total: 1min 12s
Wall time: 1min 12s


In [28]:
total_wordcount = sum(total_counter.values())
#Total number of words (not unique)
total_wordcount

6863865

In [67]:
%%time 
total_word_freq = Counter({k:v/total_wordcount for k,v in total_counter.items()})

CPU times: user 74.7 ms, sys: 16.3 ms, total: 91 ms
Wall time: 91.2 ms


In [69]:
#%%time
group_word_freq = {}
groups_freq = groups_counter
for label in train_input["output"].unique():
    group_wordcount = sum(groups_counter[label])
    group_word_freq[label] = Counter({k:v/group_wordcount for k,v in groups_freq[label].items()})

In [111]:
%%time
def get_value_dict(my_dict,key):
    if key in my_dict: 
        return my_dict[key]
    else:
        return 0
group_word_laplace = {}
total_words = len(total_counter)
for label in train_input["output"].unique():
    temp_group = groups_counter[label]
    group_wordcount = sum(groups_counter[label])
    #Conditional probability calculation with laplace smoothing
    group_word_laplace[label] = Counter({k:(get_value_dict(temp_group,k)+1)/(v + total_words + 1 )for k,v in total_counter.items()})

CPU times: user 5.33 s, sys: 622 ms, total: 5.95 s
Wall time: 6.04 s


# Predict on New Data

In [129]:
test_input = pd.read_csv('test_input.csv')
print(len(test_input))

53218


,id,conversation
0,0,<speaker_1> philadelphia is decriminalizing ma...
1,1,<speaker_1> david cameron pushes for repeal of...
2,2,"<speaker_1> just a mile from gaza , a kibbutzn..."
3,3,<speaker_1> chris christies greatest weakness ...
4,4,<speaker_1> lindt chocolat cafe hostage drama ...


In [131]:
%%time
test_input = generate_features(test_input)

CPU times: user 6.2 s, sys: 105 ms, total: 6.31 s
Wall time: 6.32 s


In [134]:
test_input.head()

,id,conversation,text,words,words_count
0,0,<speaker_1> philadelphia is decriminalizing ma...,philadelphia is decriminalizing marijuana pos...,"[philadelphia, is, decriminalizing, marijuana,...","{'latinos': 1, 'com': 1, 'happens': 1, 'hopes'..."
1,1,<speaker_1> david cameron pushes for repeal of...,david cameron pushes for repeal of u k s hu...,"[david, cameron, pushes, for, repeal, of, u, k...","{'com': 1, 'still': 1, 'u': 1, 'god': 1, 'none..."
2,2,"<speaker_1> just a mile from gaza , a kibbutzn...",just a mile from gaza a kibbutznik cartoonis...,"[just, a, mile, from, gaza, a, kibbutznik, car...","{'humanity': 1, 'hating': 1, 'gaza': 1, 'wing'..."
3,3,<speaker_1> chris christies greatest weakness ...,chris christies greatest weakness its not hi...,"[chris, christies, greatest, weakness, its, no...","{'white': 1, 'com': 1, 'matter': 1, 'temper': ..."
4,4,<speaker_1> lindt chocolat cafe hostage drama ...,lindt chocolat cafe hostage drama in martin p...,"[lindt, chocolat, cafe, hostage, drama, in, ma...","{'martin': 1, 'drama': 1, 'com': 1, 'group': 1..."


In [138]:
test_input["words_count"][1].items()

dict_items([('com', 1), ('still', 1), ('u', 1), ('god', 1), ('none', 1), ('harper', 1), ('pushes', 1), ('existent', 1), ('australia', 1), ('keep', 1), ('idiot', 1), ('rights', 1), ('britain', 1), ('around', 1), ('power', 1), ('repeal', 1), ('people', 1), ('gone', 1), ('human', 1), ('upi', 1), ('k', 1), ('elections', 1), ('canada', 2), ('fuck', 1), ('conservatives', 1), ('may', 1), ('voting', 1), ('act', 1), ('cameron', 1), ('till', 1), ('david', 1)])

In [181]:
#Now, create a function to predict class for each text snippet 

categories = train_input["output"].unique()

class_priors = {}
for category in categories:
    class_priors[category] = train_input.groupby("output").size()[category] / len(train_input)

def predict_class(word_counter):
    classes_prob = {}
    for category in categories:
        classes_prob[category] = 1
        for k, v in word_counter.items():
            classes_prob[category] *= (group_word_laplace[category][k] ** v)
        classes_prob[category] *= class_priors[category]
        #update with the prior class probability 
    return max(classes_prob, key = classes_prob.get)
            

In [197]:
%%time
test_output_series = test_input["words_count"].apply(lambda x: predict_class(x))

CPU times: user 13.5 s, sys: 960 ms, total: 14.5 s
Wall time: 14.6 s


In [198]:
test_output_series.head()

0     politics
1    worldnews
2     politics
3     politics
4    worldnews
Name: words_count, dtype: object

In [207]:
test_output = pd.DataFrame(test_input["id"])
test_output["category"] = test_output_series

In [208]:
test_output.head()

,id,category
0,0,politics
1,1,worldnews
2,2,politics
3,3,politics
4,4,worldnews


In [209]:
test_output.to_csv("naive_bayes_prediction.csv")